In [4]:
import pandas as pd
from cStringIO import StringIO

In [60]:
%%storage read --object "gs://fedwork_bigquery_exports/status_dynamic_dummies_all_agencies" --variable text

In [61]:
# Load data to dataframe
df = pd.read_csv(StringIO(text)) 
print df.shape
df.head()

(771325, 57)


,emp_id,agency_cat,occupation_cat,basic_pay,basic_pay_squared,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,...,occupation_fam_18,occupation_fam_19,occupation_fam_20,occupation_fam_21,superv_status_2,superv_status_4,superv_status_5,superv_status_6,superv_status_7,superv_status_8
0,898655,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2257068,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1745142,AA,P,113346.0,1.284732e+10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,5050496,AA,P,75621.0,5.718536e+09,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,8638776,AA,P,157100.0,2.468041e+10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [63]:
# Remove rows for small agencies
small_agencies = ['ZU','ZR','ZS','ZP','ZL','YF','YE','UT','UJ','TS','RZ','RS','RF','RE','QQ','OS','NS','NP','NM',
                  'NK','MA','IF','HW','HT','HP','HB','GY','GX','GQ','GO','GM','GG','GE','FK','FJ','FI','EW',
                  'EQ','EO','DQ','DF','DB','CX','CF','CE','CC','BZ','BW','BT','BK','AW','AP','AN','YB','JL','CG',
                 'BH','EX','IW','DG','ZN','RO','YD','ZW','IG','VD','YG','EM','EB','GI','YA','ZT','VB','PC','DA',
                 'GN','FR','ZZ','ZQ','AI', 'PU']

df = df[~df['agency_cat'].isin(small_agencies)]
print 'dataframe without individuals in small agencies', df.shape

dataframe without individuals in small agencies (770518, 57)


In [64]:
# Count null values
[[i, df[i].isnull().sum()] for i in df.columns.values]

# Remove rows with null values
# There were only null values in basic pay, individuals with null basic pay were not included in the model cox regression
df = df.dropna(axis=0, how='any')
print 'dataframe without null pay values', df.shape

dataframe without null pay values (769918, 57)


In [65]:
# Create columns for normalized basic_pay and basic_pay_squared
df['basic_pay_norm'] = df[['agency_cat','basic_pay']].groupby('agency_cat').transform(lambda x: (x - x.mean()) / x.std())
df['basic_pay_squared_norm'] = df[['agency_cat','basic_pay_squared']].groupby('agency_cat').transform(lambda x: (x - x.mean()) / x.std())
df.head()

,emp_id,agency_cat,occupation_cat,basic_pay,basic_pay_squared,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,...,occupation_fam_20,occupation_fam_21,superv_status_2,superv_status_4,superv_status_5,superv_status_6,superv_status_7,superv_status_8,basic_pay_norm,basic_pay_squared_norm
0,898655,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.158587,1.208730
1,2257068,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1.158587,1.208730
2,1745142,AA,P,113346.0,1.284732e+10,0,0,0,1,0,...,0,0,0,0,0,0,0,1,-0.296075,-0.397890
3,5050496,AA,P,75621.0,5.718536e+09,0,0,0,1,0,...,0,0,0,0,0,0,0,1,-1.433330,-1.261162
4,8638776,AA,P,157100.0,2.468041e+10,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.022931,1.035060


In [66]:
df.tail()

,emp_id,agency_cat,occupation_cat,basic_pay,basic_pay_squared,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,...,occupation_fam_20,occupation_fam_21,superv_status_2,superv_status_4,superv_status_5,superv_status_6,superv_status_7,superv_status_8,basic_pay_norm,basic_pay_squared_norm
771319,2797409,VA,P,309439.0,9.575249e+10,0,0,0,0,0,...,0,0,0,0,0,0,0,1,4.727701,6.442418
771320,8292536,VA,P,309439.0,9.575249e+10,0,0,0,0,0,...,0,0,0,0,0,0,0,1,4.727701,6.442418
771321,244892,VA,P,309695.0,9.591099e+10,0,0,0,0,0,...,0,0,1,0,0,0,0,0,4.732965,6.454146
771322,2263970,VA,P,277311.0,7.690139e+10,0,0,0,0,0,...,0,0,1,0,0,0,0,0,4.067028,5.047472
771323,2881482,VA,P,277695.0,7.711451e+10,0,0,0,0,0,...,0,0,1,0,0,0,0,0,4.074924,5.063242


In [67]:
# Check that normalizations are correct
mAA = df[df['agency_cat'] == 'AA']['basic_pay'].mean()
sdAA = df[df['agency_cat'] == 'AA']['basic_pay'].std()

m2AA = df[df['agency_cat'] == 'AA']['basic_pay_squared'].mean()
sd2AA = df[df['agency_cat'] == 'AA']['basic_pay_squared'].std()

print df.iloc[0]['basic_pay_norm'] , (df.iloc[0]['basic_pay'] - mAA) / sdAA
print df.iloc[0]['basic_pay_squared_norm'] , (df.iloc[0]['basic_pay_squared'] - m2AA) / sd2AA

mVA = df[df['agency_cat'] == 'VA']['basic_pay'].mean()
sdVA = df[df['agency_cat'] == 'VA']['basic_pay'].std()

m2VA = df[df['agency_cat'] == 'VA']['basic_pay_squared'].mean()
sd2VA = df[df['agency_cat'] == 'VA']['basic_pay_squared'].std()

print df.iloc[-1]['basic_pay_norm'] , (df.iloc[-1]['basic_pay'] - mVA) / sdVA
print df.iloc[-1]['basic_pay_squared_norm'] , (df.iloc[-1]['basic_pay_squared'] - m2VA) / sd2VA

1.1585873474 1.1585873474
1.20873048814 1.20873048814
4.07492422455 4.07492422455
5.06324219842 5.06324219842


In [68]:
# Check for any null values
[[i, df[i].isnull().sum()] for i in df.columns.values]

[['emp_id', 0],
 ['agency_cat', 0],
 ['occupation_cat', 0],
 ['basic_pay', 0],
 ['basic_pay_squared', 0],
 ['age_15_19', 0],
 ['age_20_24', 0],
 ['age_25_29', 0],
 ['age_30_34', 0],
 ['age_35_39', 0],
 ['age_40_44', 0],
 ['age_45_49', 0],
 ['age_50_54', 0],
 ['age_55_59', 0],
 ['age_60_64', 0],
 ['age_65_69', 0],
 ['age_70_74', 0],
 ['age_75', 0],
 ['age_UNSP', 0],
 ['los_1', 0],
 ['los_1_2', 0],
 ['los_3_4', 0],
 ['los_5_9', 0],
 ['los_10_14', 0],
 ['los_15_19', 0],
 ['los_20_24', 0],
 ['los_25_29', 0],
 ['los_30_34', 0],
 ['los_35', 0],
 ['los_UNSP', 0],
 ['occupation_fam_01', 0],
 ['occupation_fam_02', 0],
 ['occupation_fam_03', 0],
 ['occupation_fam_04', 0],
 ['occupation_fam_05', 0],
 ['occupation_fam_06', 0],
 ['occupation_fam_07', 0],
 ['occupation_fam_08', 0],
 ['occupation_fam_09', 0],
 ['occupation_fam_10', 0],
 ['occupation_fam_11', 0],
 ['occupation_fam_12', 0],
 ['occupation_fam_13', 0],
 ['occupation_fam_14', 0],
 ['occupation_fam_15', 0],
 ['occupation_fam_16', 0],
 ['oc

In [69]:
# Shape of final df for tableau
df.shape

(769918, 59)

In [71]:
# Write to file
df.to_csv('norm_data.csv', sep=',')